<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Topic Modeling and Latent Dirichlet Allocation (LDA)

_Authors: Dave Yerrington (SF)_

---

### Learning Objectives
- See how search engines benefit from topic modeling.
- Visualize a simple example of term vectors.
- Walk through the coding of a simplified LDA topic model.
- Gain an intuition for what LDA is doing.
- Understand the pros and cons to LDA topic modeling.

### Lesson Guide
- [Introduction: search engines and topic modeling](#intro)
- [Term vector model visual example](#term-vector-ex)
- [How LDA works](#how-lda-works)
    - [Step 1: choose K topics](#step1)
    - [Step 2: randomly assign words in documents to topics](#step2)
    - [Step 3: word distributions of topics](#step3)
    - [Step 4: reassignment of topics](#step4)
    - [Step 5: getting words that define topics and topics that define documents](#step5)
    - [Major caveat: oversimplified!](#caveat)
- [LDA intuition](#lda-intuition)
- [LDA challenges](#lda-challenges)
- [LDA strengths](#lda-strengths)
- [Other models similar to LDA](#other-models)

<a id='intro'></a>
## Introduction: search engines and topic modeling
---

Search Engines use a variety of natural language processing techniques to provide an accurate query interface to documents on the internet.  Latent Dirichlet Allocation, which we will be discussing in this lecture, is just one of the many tools employed to improve search engine experience.



![](https://snag.gy/lbsuV2.jpg)

![](https://snag.gy/YdgxKz.jpg)

![](https://snag.gy/Ctr6OL.jpg)

![](https://snag.gy/ob9Um8.jpg)

<a id='term-vector-ex'></a>

## Term vector model visual example

---

The code below plots a basic example of differentiating between topics using *vectors*. Specifically, the similarity between vectors can be used to infer the similarity or dissimilarity between topics. 

In [1]:
%matplotlib inline

In [1]:
##### from scipy.interpolate import spline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Wedge
import matplotlib.patches as patches
from matplotlib.path import Path
import pandas as pd

fig = plt.figure(figsize=(8, 8))
ax = plt.gca()
theta1, theta2 = 0, 90
radius = .1
center = (0, 0)

# Main angle guide
w1 = Wedge(center, radius, theta1, theta2, fill=False, linestyle='dashed')
w1.set_linewidth(1)
ax.add_artist(w1)

# Vector examples
ax.annotate('Dave', (0, 0), (0.5, 0.5),
            arrowprops=dict(arrowstyle='<-', color="g"),
            xycoords='data', textcoords='axes fraction', color="g")

ax.annotate('Bigfoot', (0, 0), (0.35, 0.4),
            arrowprops=dict(arrowstyle='<-', color="g"),
            xycoords='data', textcoords='axes fraction', color="g")

ax.annotate('Carrillo', (0, 0), (0.2, 0.3),
            arrowprops=dict(arrowstyle='<-', color="g"),
            xycoords='data', textcoords='axes fraction', color="g")

ax.annotate('Calrissian', (0, 0), (0.15, 0.63),
            arrowprops=dict(arrowstyle='<-', color="b", linewidth=2.5),
            xycoords='data', textcoords='axes fraction', color="b")

ax.annotate('Lando', (0, 0), (0.05, 0.6),
            arrowprops=dict(arrowstyle='<-', color="b", linewidth=2.5),
            xycoords='data', textcoords='axes fraction', fontsize=12, color="b")

ax.annotate('Spock', (0, 0), (.6, 0.09),
            arrowprops=dict(arrowstyle='<-', color="r", linewidth=2.5),
            xycoords='data', textcoords='axes fraction', color="r")

ax.annotate('Picard', (0, 0), (.7, 0.025),
            arrowprops=dict(arrowstyle='<-', linewidth=2.5, color="r"),
            xycoords='data', textcoords='axes fraction', color="r")

# Remove splines on top and right
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['top'].set_visible(False)
# ax.spines['bottom'].set_visible(False)
# ax.get_xaxis().set_visible(False)
# ax.get_yaxis().set_visible(False)
ax.set_ylim([0, .25])
ax.set_xlim([0, .25])
ax.set_xmargin(.001)
ax.set_ymargin(.001)
ax.set_autoscale_on(False)


# x = np.linspace(0, 2*np.pi, 100)
# plt.plot(x)
plt.title("Basic Term Vector Model", fontsize=25, position=(0.5, 0.8))
plt.ylabel("Star Wars", fontsize=18, color="b")
plt.xlabel("Star Track", fontsize=18, color="r")


plt.margins(0.005, 0.0001)

plt.show()

<Figure size 800x800 with 1 Axes>

Each word in our corpus is related to either "Star Wars" or "Star Track".  We use a vector space model to calculate the distance of each word to each topic.

> Topics generated from an LDA model are actually a cluster of word probabilities, not clearly defined labels.  Simplifying word vectors like this, should give you a sense about the intuition of how **words vectors** relate to **topics**.

<a id='how-lda-works'></a>
## How LDA Works

---

_Abridged Explanation_

![](https://snag.gy/aiSFrm.jpg)

LDA isn't exactly straightforward when it comes to the math.  But we can explore this piecewise in more general terms.

In [2]:
import pandas as pd

yelp = pd.read_csv('../datasets/yelp.csv', encoding='utf-8')

In [3]:
yelp.head()

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0


In [4]:
yelp['len'] = yelp['text'].map(lambda t: len(t))

In [5]:
#very naive method!
yelp['words'] = yelp['text'].map(lambda t: len(t.replace('.',' ').split(' ')))

In [6]:
yelp.head()

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny,len,words
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0,889,171
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0,1345,274
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0,76,17
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0,419,79
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0,469,102


In [7]:
from textblob import TextBlob

In [8]:
from sklearn.feature_extraction import stop_words
stopwords = stop_words.ENGLISH_STOP_WORDS

** Split the reviews into good (5 stars) and bad (1 star) **

In [9]:
good = [list(TextBlob(x.lower()).words) for x in yelp[yelp['stars']==5].text.values[0:25]]
bad = [list(TextBlob(x.lower()).words) for x in yelp[yelp['stars']==1].text.values[0:25]]

In [10]:
good = [[w for w in doc if w not in stopwords] for doc in good]
bad = [[w for w in doc if w not in stopwords] for doc in bad]
good[11]

['time',
 'friend',
 'went',
 'delicious',
 'food',
 'garlic',
 'knots',
 'favorite',
 'course',
 'wine',
 'going',
 'alot']

<a id='step1'></a>
### Step 1: choose K topics

Similar to the KNN algorithm. By setting K we are deciding up front on a preset number of topics to determine.

In [11]:
K = 5

<a id='step2'></a>
### Step 2: randomly assign words in documents to topics

First let's denote the symbols for the different components:

$D$ represents the documents, $d$ is a document.

$W$ represents all the words in the documents, $w$ is a word.

$Z$ represents the collection of our $K$ topics, and $z$ is one of those topics.

We will start off with a completely random assignment of words to topics. Iterate through the documents $D$ and for each word $W$ in each document randomly assign the word to be in one of the $Z$ topics.

In [13]:
# get the documents and all the words:
D = good+bad
W = [w for wordlist in D for w in wordlist]
print(len(D), len(W))

50 2676


In [14]:
# get the UNIQUE words:
W_unique = list(np.unique(W))
print(len(W_unique))

1410


In [15]:
# Make the topics. Each topic will have a dictionary of counts of occurrances
# of each word, which we will set to zero for now:
Z = {i:{w:0 for w in W_unique} for i in range(K)}

In [16]:
Z.keys()

dict_keys([0, 1, 2, 3, 4])

In [17]:
Z[0]

{"'d": 0,
 "'ll": 0,
 "'m": 0,
 "'re": 0,
 "'s": 0,
 "'ve": 0,
 '1': 0,
 '1-1/2': 0,
 '1/2': 0,
 '1/3': 0,
 '10': 0,
 '100': 0,
 '11': 0,
 '115': 0,
 '15': 0,
 '150': 0,
 '16th': 0,
 '1pm': 0,
 '2': 0,
 '2.50': 0,
 '20': 0,
 '20-38': 0,
 '20mbs': 0,
 '25': 0,
 '2:05': 0,
 '2nd': 0,
 '3.79': 0,
 '3/4': 0,
 '30': 0,
 '39': 0,
 '4': 0,
 '40': 0,
 '45': 0,
 '5': 0,
 '5-20': 0,
 '5-6': 0,
 '5:50': 0,
 '5:52': 0,
 '6': 0,
 '60': 0,
 '64': 0,
 '6:02': 0,
 '6:42': 0,
 '6pm': 0,
 '70': 0,
 '7:30': 0,
 '7pm': 0,
 '7th': 0,
 '8': 0,
 '80': 0,
 '8:15': 0,
 '9': 0,
 '9:25': 0,
 'absolute': 0,
 'absolutely': 0,
 'abstract': 0,
 'absurd': 0,
 'accepted': 0,
 'accident': 0,
 'according': 0,
 'action': 0,
 'add': 0,
 'addition': 0,
 'adequate': 0,
 'adequately': 0,
 'admit': 0,
 'adorable': 0,
 'adult': 0,
 'advertised': 0,
 'aged': 0,
 'ages': 0,
 'ahead': 0,
 'ahi': 0,
 'aid': 0,
 'aj': 0,
 'albeit': 0,
 'allow': 0,
 'alot': 0,
 'alphagraphics': 0,
 'alright': 0,
 'alteration': 0,
 'alternatives': 0,

In [18]:
# go through all the words in all the documents, and randomly assign each
# one to one of the topics:
D_z = [[[w, np.random.choice(range(K))] for w in d] for d in D]
D_z[0]

[['wife', 4],
 ['took', 3],
 ['birthday', 0],
 ['breakfast', 1],
 ['excellent', 0],
 ['weather', 0],
 ['perfect', 2],
 ['sitting', 4],
 ['outside', 4],
 ['overlooking', 0],
 ['grounds', 1],
 ['absolute', 4],
 ['pleasure', 3],
 ['waitress', 0],
 ['excellent', 2],
 ['food', 0],
 ['arrived', 1],
 ['quickly', 2],
 ['semi-busy', 2],
 ['saturday', 0],
 ['morning', 4],
 ['looked', 3],
 ['like', 4],
 ['place', 2],
 ['fills', 3],
 ['pretty', 1],
 ['quickly', 2],
 ['earlier', 4],
 ['better', 3],
 ['favor', 1],
 ['bloody', 2],
 ['mary', 3],
 ['phenomenal', 2],
 ['simply', 1],
 ['best', 3],
 ["'ve", 2],
 ["'m", 1],
 ['pretty', 0],
 ['sure', 4],
 ['use', 3],
 ['ingredients', 2],
 ['garden', 1],
 ['blend', 2],
 ['fresh', 3],
 ['order', 0],
 ['amazing', 3],
 ['menu', 0],
 ['looks', 0],
 ['excellent', 4],
 ['white', 2],
 ['truffle', 3],
 ['scrambled', 0],
 ['eggs', 3],
 ['vegetable', 2],
 ['skillet', 4],
 ['tasty', 2],
 ['delicious', 0],
 ['came', 0],
 ['2', 2],
 ['pieces', 0],
 ['griddled', 0],
 ['br

In [19]:
# update the topic dictionaries:
for d in D_z:
    for w, z in d:  
        Z[z][w] += 1

<a id='step3'></a>
### Step 3: word distributions of topics

Now that we have randomly assigned words to topics, we have the word distributions of different topics. These distributions indicate the probability of a word being in a topic. We can write out the probability of some given word being selected given a choice of one of our topics:

### $$ P(\text{word } w \;|\; \text{topic } z) $$

We also have the probability of *topic* occurance given a specific document, which is the proportion of words in a document that are assigned to a given topic.

### $$ P(\text{topic } z \;|\; \text{document } d) $$


In [20]:
# p(z|d): proportion of words in document d assigned to topic z
def topic_given_doc(z_ind, d):
    return np.sum([w_z[1] == z_ind for w_z in d])/float(len(d))

In [31]:
# p(w|z): proportion of words in topic z that are this word
def word_given_topic(w, z):
    total_words = np.sum(list(z.values()))
    if total_words == 0:
        return 0.
    else:
        return z[w]/float(total_words)

In [32]:
topic_given_doc(3, D_z[0])

0.20833333333333334

In [33]:
word_given_topic('wife', Z[1])

0.0

<a id='step4'></a>
### Step 4: reassignment of topics

Now we will go through an iterative procedure. We will walk through the words in the documents many times. At each step we will calculate which topic a word should be reassigned to.

First we neet to calculate the probability associated with each topic:

### $$ P(\text{topic } z \;|\; \text{document } d) \cdot P(\text{word } w \;|\; \text{topic } z) $$

Which is the probability that the topic generated our current word. Below we can calculate the probability of a word in the first document being generated from each of our topics:

In [34]:
for z_ind in range(K):
    print('topic:', z_ind, 'P:', topic_given_doc(z_ind, D_z[0]) * word_given_topic('wife', Z[z_ind]))

topic: 0 P: 0.0
topic: 1 P: 0.0
topic: 2 P: 0.0
topic: 3 P: 0.0
topic: 4 P: 0.00028825995807127885


We can write a function that will sample from the topics according to these probabilities to assign a new topic:

In [35]:
def sample_topic(Z, document, word):
    probs = []
    for i in range(len(Z)):
        probs.append(topic_given_doc(i, document) * word_given_topic(word, Z[i]))
    if np.sum(probs) == 0:
        return -1
    return np.random.choice(range(len(Z)), p=np.array(probs)/np.sum(probs))

In [36]:
sample_topic(Z, D_z[0], 'wife')

4

To put it all together, we can write a a function that will iterate over all the words/documents a specified number of times and do the reassignment:

In [37]:
# WARNING: SLOW!
def topic_iterator(Z, D_z, iters=20):
    for it in range(iters):
        print('iter:', it)
        for d_ind in range(len(D_z)):
            for w_ind in range(len(D_z[d_ind])):
                old_topic = D_z[d_ind][w_ind][1]
                word = D_z[d_ind][w_ind][0]
                new_topic = sample_topic(Z, D_z[d_ind], word)
                if new_topic != -1:
                    Z[old_topic][word] = max(0, Z[old_topic][word]-1)
                    Z[new_topic][word] = Z[new_topic][word]+1

In [38]:
# WARNING: SLOW!
topic_iterator(Z, D_z)

iter: 0
iter: 1
iter: 2
iter: 3
iter: 4
iter: 5
iter: 6
iter: 7
iter: 8
iter: 9
iter: 10
iter: 11
iter: 12
iter: 13
iter: 14
iter: 15
iter: 16
iter: 17
iter: 18
iter: 19


<a id='step5'></a>
### Step 5: getting words that define topics and topics that define documents

To get the topic distribution for a document we can use the function that calculated the probability of a topic given a document from before.

In [39]:
def topic_dist(document, K=5):
    for z in range(K):
        print('topic:', z, 'P:', topic_given_doc(z, document))

Let's check it out for the first 3 documents:

In [40]:
for d, doc in enumerate(D_z[0:3]):
    print('document:', d)
    topic_dist(doc)
    print('-'*40)

document: 0
topic: 0 P: 0.2361111111111111
topic: 1 P: 0.1388888888888889
topic: 2 P: 0.2638888888888889
topic: 3 P: 0.20833333333333334
topic: 4 P: 0.1527777777777778
----------------------------------------
document: 1
topic: 0 P: 0.26
topic: 1 P: 0.25
topic: 2 P: 0.16
topic: 3 P: 0.14
topic: 4 P: 0.19
----------------------------------------
document: 2
topic: 0 P: 0.22727272727272727
topic: 1 P: 0.13636363636363635
topic: 2 P: 0.09090909090909091
topic: 3 P: 0.25
topic: 4 P: 0.29545454545454547
----------------------------------------


To get the words that define topics we can just calculate the words that have greatest probability of occuring for a given topic

In [43]:
def topic_top_words(z, top=5):
    word_sum = np.sum(list(z.values()))
    word_counts = sorted(z.items(), key=lambda x: x[1], reverse=True)
    top_words = [[w, c/float(word_sum)] for w, c in word_counts[0:top]]
    for w, p in top_words:
        print(w, p)

In [44]:
for z_ind in range(K):
    print('topic:', z_ind)
    topic_top_words(Z[z_ind])
    print('-'*40)

topic: 0
n't 0.14451754385964913
place 0.06491228070175438
's 0.06030701754385965
like 0.04780701754385965
really 0.03618421052631579
----------------------------------------
topic: 1
've 0.06719563600297546
food 0.051078601537317136
try 0.03074634267294818
staff 0.029506570790974462
customer 0.026779072650632285
----------------------------------------
topic: 2
good 0.06478149100257069
just 0.04498714652956298
sandwich 0.0390745501285347
's 0.03496143958868895
'm 0.03470437017994859
----------------------------------------
topic: 3
time 0.0672353236327145
service 0.05845459106874059
bad 0.05268439538384345
way 0.039136979427997994
people 0.037380832915203215
----------------------------------------
topic: 4
best 0.058557486312782674
chicken 0.04356105689121638
pizza 0.03594382289930969
said 0.03499166865032135
did 0.03023089740537967
----------------------------------------


<a id='caveat'></a>
### Major caveat: oversimplified!

This is a dumb, oversimplified version of LDA. We've completely ignored priors and didn't talk about the Dirichlet distribution at all. That being said, this is the general breakdown of whats going on. We perform a procedure where we assign words to topics based on how likely a topic would have generated that word. 

> **Note:** More legitimate implementations of LDA have hyperparameters such as alpha. Alpha is a scaler that helps minimize an error term.  Thankfully, most LDA models that are implented will set this automatically and it's usually, 95% of the time a fine solution.  To really get a strong handle of the math behind this model, there are whitepapers you can read.  Also, having a strong handle on Bayesian statistic is a must to really grasp this model at it's lowest levels.  We are not going there today!


<a id='lda-intuition'></a>
## LDA Intuition

---
 
As we iterate through each word in our corpus and (re)assign it to a topic:

1. Words become more common in topics where they are already common.
1. Topics will become more common in documents where they are already common.

**Remember:**:
- Words are assigned to topics randomly at first
- As words are found to be consistently distributed within topics, the model achieves a sort of equilibrium based on the distribution of words accross all documents.

<a id='lda-challenges'></a>
## LDA challenges

---

1. **There's a bit of entropy to topics.**
There can be between 1-10% shift in what is generated in LDA models.  You may not get the same thing 2x!

1. **Can be very difficult to assess.**
If you have a large corpus, with many topics (>10), it's damn near impossible to visualize the distribution of documents to topics.

1. **Preprocessing is heavy.**
To get the most out of LDA, cleaning stopwords, and specific language can be a challenging task.  Sometimes it's difficult to avoid the noise involved with this model.

1. **SME is necessary for accurate topic assessment.**
The more straight forward your text is, the less subject matter expertise is required.  A more advanced use of LDA would involve assessing documents with lots of idiomtic language. Knowing what topics are found, can be subjective.

1.  **Determining what topics mean is tricky.**
A collection of world probabiltiies generally isn't very intuitive.  You could take the first word and use that as your topic "label".  Hence, more subject matter expertise is required.

1. **LDA is unsupervised.**
It's not possible to know what is "correct".  The repsonse topics are generated. LDA is known as a "generative" model. 

1. **Tuning your LDA model can be tough."**
It's possible to tune for the parameter **K** *number of topics*, but it's not necessarily a very straightforward way to improve your model.



<a id='lda-strengths'></a>
## LDA strengths

---

1. **It can be very strong performer in production.**
After you build the model, it can easily be used "online".
> "Online" training allows you to update your model with more training data without having to refit all your data.  Only new data can be fit globally.

1.  **It's easy to get a quick sense of what a large body of text is broadly "about", without having to read all of it.** Rather than reading 12k PDF's on corproate policies, you could extract the text, and run LDA to see what generalities it finds.
1.  **Easily classify / tag documents by topic.**
1.  **It can "just work" out of the box.**  However, your mileage will vary depending on your preprocessing.

<a id='other-models'></a>
## Other models similar to LDA

---

- Topics Over Time
- Dynamic Topic Modeling
- Hierarchical LDA
- Pachinko Allocation 

A cool new LDA model to look out for:

- [LDA2Vec](http://multithreaded.stitchfix.com/blog/2016/05/27/lda2vec/)